# Circuits and Programs

In CK the circuit and program functionality are directly available, independently of a PGM.

## Circuits

In [1]:
from ck.circuit import Circuit

cct = Circuit()

x0 = cct.new_var()  # this var will have index 0
x1 = cct.new_var()  # this var will have index 1
c123 = cct.const(123)
m = cct.mul(x0, x1)
a = cct.add(c123, m)

Every circuit variable has an index, starting from zero and counting in the order that the variables were created.

In [2]:
print(x0.idx, x1.idx)

0 1


Circuit variables can be accessed from the circuit by their index.

In [3]:
cct.vars[0]

In [4]:
x0 == cct.vars[0]

True

In [5]:
x0 == cct.vars[1]

False

A circuit details can be dumped to a human-readable form.

Op nodes are printed in the form: {operation}<{ref-number}>: {arguments}

Var nodes are printed in the form: var[\{idx\}]

Const nodes are printed merely as constant values.


In [6]:
cct.dump()

number of vars: 2
number of const nodes: 3
number of op nodes: 2
number of operations: 2
number of arcs: 4
var nodes: 2
const nodes: 3
  0
  1
  123
op nodes: 2 (arcs: 4, ops: 2)
  add<1>: 123 mul<0>
  mul<0>: var[0] var[1]


## Programs

A circuit can be compiled to a program by using a `CircuitCompiler`. A default circuit compiler is
available as `ck.circuit_compiler.DEFAULT_CIRCUIT_COMPILER`. A circuit compiler
takes a collection of `CircuitNode` objects and returns a `RawProgram` to calculate the values of the given circuit nodes.

The following example compiles the circuit above.

In [7]:
from ck.circuit_compiler import DEFAULT_CIRCUIT_COMPILER
from ck.program import RawProgram


raw_program: RawProgram = DEFAULT_CIRCUIT_COMPILER(a)  # `a` is the top "addition" node of the circuit.

Every program has a numpy `dtype` which defines the numeric data type for input and output values.

In this case, the default type was used, which is a C style double.

In [8]:
raw_program.dtype

ctypes.c_double

The raw program is a function from an input array to an output array.

In [9]:
print(raw_program([0, 1]))

[123.]


a raw program is typically referenced and used by other objects. For example a raw program can be wrapped in a `ProgramBuffer` which provides pre-allocated buffers for input, output, and temporary values for more efficient and convenient computations.

In [10]:
from ck.program import ProgramBuffer

program = ProgramBuffer(raw_program)

program[0] = 123   # set the first program argument
program[1] = 456   # set the second program argument


print(program[0], program[1])


123.0 456.0


The result of a program buffer is obtained by calling `compute`.

In [11]:
program.compute()

array([56211.])

The last computed results are always available in the results buffer.

In [12]:
program.results

array([56211.])

In general, a program can have multiple results, which is why results are available as a numpy array.

The following example returns two values. These are from circuit nodes `m` and `a`.

In [13]:
program2 = ProgramBuffer(DEFAULT_CIRCUIT_COMPILER(m, a))

In [14]:
program2.number_of_results

2

In [15]:
program2[0] = 30
program2[1] = 50

In [16]:
program2.compute()

array([1500., 1623.])

Whether a program is producing a single result or an array of results, the results are always available as an array, using 'results'.

In [17]:
program.results

array([56211.])

In [18]:
program2.results

array([1500., 1623.])

A program buffer is not thread safe because access to the buffers is unprotected. However, you can easily clone a program buffer which uses the same raw program but different memory allocations for buffers. Consequently, cloned program buffers can be safely set and computed in different threads.

In [19]:
program3 = program2.clone()

program3.compute()

array([1500., 1623.])